# Alpaca-py options trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

api_key = "PKPSYNLLG60LVBQIF1Q7"
secret_key = "kA2toSKRsStgvvqZ48qTbb6aaYhjuLVgZfl1gXmh"


#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [2]:
# install alpaca-py
! python3 -m pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 8.0 MB/s eta 0:00:00


In [33]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [34]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [35]:
# check version of alpaca-py
alpaca.__version__

'0.33.0'

# Trading Client

In [36]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [37]:
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA30QIJYEPG7',
    'accrued_fees': '0',
    'buying_power': '2452541.18',
    'cash': '-741972.37',
    'created_at': datetime.datetime(2023, 3, 23, 20, 23, 14, 889110, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 4,
    'daytrading_buying_power': '2452541.18',
    'equity': '1225892.2',
    'id': UUID('2b83dd79-e5f9-4c51-adbe-25b81d943e0f'),
    'initial_margin': '983932.29',
    'last_equity': '1219809.14',
    'last_maintenance_margin': '605455.45',
    'long_market_value': '1967864.57',
    'maintenance_margin': '590359.37',
    'multiplier': '4',
    'non_marginable_buying_power': '239959.91',
    'options_approved_level': 2,
    'options_buying_power': '635532.83',
    'options_trading_level': 2,
    'pattern_day_trader': True,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '1225892.2',
    'regt_buying_power':

In [38]:
# check account configuration
# - we have new field `max_options_trading_level`
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [39]:
# get list of assets which are options enabled
# - we can filter assets by `options_enabled` attribute
# - asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('0a42abd1-23be-489f-9a2b-03014ca6633e'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'ARRIVAL Common Shares',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'ARVLF',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('8332396f-6baf-49b1-8717-e51da871b4a9'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'BETTER THERAPEUTICS I

In [40]:
# get list of options contracts for the given underlying symbol (e.g. SPY,AAPL)
# - get_option_contracts() is a new method to get list of options contracts
# - in this example, we get 2 options contracts for SPY,AAPL
# - you can continue to fetch options contracts by specifying page_token from next_page_token of response
underlying_symbols = ["SPY", "AAPL"]
req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,               # specify underlying symbols
    status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
    expiration_date = None,                                # specify expiration date (specified date + 1 day range)
    expiration_date_gte = None,                            # we can pass date object
    expiration_date_lte = None,                            # or string (YYYY-MM-DD)
    root_symbol = None,                                    # specify root symbol
    type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
    style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
    strike_price_gte = None,                               # specify strike price range
    strike_price_lte = None,                               # specify strike price range
    limit = 2,                                             # specify limit
    page_token = None,                                     # specify page token
)
res = trade_client.get_option_contracts(req)
res

{   'next_page_token': 'Mg==',
    'option_contracts': [   {   'close_price': '126.98',
                                'close_price_date': datetime.date(2024, 11, 18),
                                'expiration_date': datetime.date(2024, 11, 22),
                                'id': '9129e4ed-047d-46d4-b4d4-253d635ced80',
                                'name': 'AAPL Nov 22 2024 100 Call',
                                'open_interest': '9',
                                'open_interest_date': datetime.date(2024, 11, 15),
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 100.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL241122C00100000',
                                'tradable': True,
                                'type': <C

In [41]:
# continue to fetch option contracts if there is next_page_token in response
if res.next_page_token is not None:
    req = GetOptionContractsRequest(
        underlying_symbols = underlying_symbols,               # specify underlying symbols
        status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
        expiration_date = None,                                # specify expiration date (specified date + 1 day range)
        expiration_date_gte = None,                            # we can pass date object
        expiration_date_lte = None,                            # or string (YYYY-MM-DD)
        root_symbol = None,                                    # specify root symbol
        type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
        style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
        strike_price_gte = None,                               # specify strike price range
        strike_price_lte = None,                               # specify strike price range
        limit = 2,                                             # specify limit
        page_token = res.next_page_token,                      # specify page token
    )
    res = trade_client.get_option_contracts(req)
    display(res)

{   'next_page_token': 'NA==',
    'option_contracts': [   {   'close_price': None,
                                'close_price_date': None,
                                'expiration_date': datetime.date(2024, 11, 22),
                                'id': '2d3470b2-2f67-4d64-b61f-f36ef30e9118',
                                'name': 'AAPL Nov 22 2024 110 Call',
                                'open_interest': None,
                                'open_interest_date': None,
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 110.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL241122C00110000',
                                'tradable': True,
                                'type': <ContractType.CALL: 'call'>,
                      

In [42]:
# get options contract by symbol
# - get_option_contract() is a new method to get options contract by symbol or id
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

{   'close_price': None,
    'close_price_date': None,
    'expiration_date': datetime.date(2024, 11, 22),
    'id': '2d3470b2-2f67-4d64-b61f-f36ef30e9118',
    'name': 'AAPL Nov 22 2024 110 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 110.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL241122C00110000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [43]:
# get options contract by id
id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=id)
contract

{   'close_price': None,
    'close_price_date': None,
    'expiration_date': datetime.date(2024, 11, 22),
    'id': '2d3470b2-2f67-4d64-b61f-f36ef30e9118',
    'name': 'AAPL Nov 22 2024 110 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 110.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL241122C00110000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [44]:
# get put options contracts
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)
res.option_contracts[:2]

[{   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 11, 20),
     'id': 'f137e120-cc28-48aa-a0d2-4bac5a513d2b',
     'name': 'SPY Nov 20 2024 300 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 300.0,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'SPY241120P00300000',
     'tradable': True,
     'type': <ContractType.PUT: 'put'>,
     'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'underlying_symbol': 'SPY'},
 {   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 11, 20),
     'id': '2905e1c3-b560-40c8-b3ec-e346e2c0eef4',
     'name': 'SPY Nov 20 2024 305 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,


In [45]:
# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

{   'close_price': '0.29',
    'close_price_date': datetime.date(2024, 11, 18),
    'expiration_date': datetime.date(2024, 11, 20),
    'id': '64a72a00-1651-4db4-a6b0-c4ba6525d3b4',
    'name': 'SPY Nov 20 2024 577 Put',
    'open_interest': '6400',
    'open_interest_date': datetime.date(2024, 11, 15),
    'root_symbol': 'SPY',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 577.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'SPY241120P00577000',
    'tradable': True,
    'type': <ContractType.PUT: 'put'>,
    'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'underlying_symbol': 'SPY'}

In [46]:
# place buy put option order
# - we can place buy put option order same as buy stock/crypto order
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 1,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('64a72a00-1651-4db4-a6b0-c4ba6525d3b4'),
    'canceled_at': None,
    'client_order_id': 'faf259f9-e7eb-4ee4-97bc-ecfa639e095c',
    'created_at': datetime.datetime(2024, 11, 19, 16, 28, 32, 187421, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('f9c8e809-f18d-45df-8584-532c68a0ca89'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 11, 19, 16, 28, 32, 185711, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY

In [47]:
# get list of orders by specifying option contract symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [high_open_interest_contract.symbol],
    limit = 2,
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
     'asset_id': UUID('64a72a00-1651-4db4-a6b0-c4ba6525d3b4'),
     'canceled_at': None,
     'client_order_id': 'faf259f9-e7eb-4ee4-97bc-ecfa639e095c',
     'created_at': datetime.datetime(2024, 11, 19, 16, 28, 32, 187421, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': datetime.datetime(2024, 11, 19, 16, 28, 32, 247419, tzinfo=TzInfo(UTC)),
     'filled_avg_price': '0.2',
     'filled_qty': '1',
     'hwm': None,
     'id': UUID('f9c8e809-f18d-45df-8584-532c68a0ca89'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.MARKET: 'market'>,
     'qty': '1',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.FILLED: 'filled'>,
     'stop_price': None,
     'submitted_at': datetim

In [48]:
# below cells should be done after market open otherwise there is no position for the option contract

# get positions filtered by option contract symbol
# if you do this example outside of market hours, you will see empty list
# because we have no position in this option contract
# please wait market open and run this example again
positions = trade_client.get_all_positions()
[pos for pos in positions if pos.symbol == high_open_interest_contract.symbol]

[{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
     'asset_id': UUID('64a72a00-1651-4db4-a6b0-c4ba6525d3b4'),
     'asset_marginable': True,
     'avg_entry_price': '0.2',
     'avg_entry_swap_rate': None,
     'change_today': '-0.2758620689655172',
     'cost_basis': '20',
     'current_price': '0.21',
     'exchange': <AssetExchange.EMPTY: ''>,
     'lastday_price': '0.29',
     'market_value': '21',
     'qty': '1',
     'qty_available': '1',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'SPY241120P00577000',
     'unrealized_intraday_pl': '1',
     'unrealized_intraday_plpc': '0.05',
     'unrealized_pl': '1',
     'unrealized_plpc': '0.05',
     'usd': None}]

In [25]:
# get positions by symbol
trade_client.get_open_position(symbol_or_asset_id=high_open_interest_contract.symbol)


{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('64a72a00-1651-4db4-a6b0-c4ba6525d3b4'),
    'asset_marginable': True,
    'avg_entry_price': '0.23',
    'avg_entry_swap_rate': None,
    'change_today': '-0.3103448275862069',
    'cost_basis': '23',
    'current_price': '0.2',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.29',
    'market_value': '20',
    'qty': '1',
    'qty_available': '1',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY241120P00577000',
    'unrealized_intraday_pl': '-3',
    'unrealized_intraday_plpc': '-0.1304347826086957',
    'unrealized_pl': '-3',
    'unrealized_plpc': '-0.1304347826086957',
    'usd': None}

In [26]:
# get positions by contract id
trade_client.get_open_position(symbol_or_asset_id = high_open_interest_contract.id)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('64a72a00-1651-4db4-a6b0-c4ba6525d3b4'),
    'asset_marginable': True,
    'avg_entry_price': '0.23',
    'avg_entry_swap_rate': None,
    'change_today': '-0.3103448275862069',
    'cost_basis': '23',
    'current_price': '0.2',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.29',
    'market_value': '20',
    'qty': '1',
    'qty_available': '1',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY241120P00577000',
    'unrealized_intraday_pl': '-3',
    'unrealized_intraday_plpc': '-0.1304347826086957',
    'unrealized_pl': '-3',
    'unrealized_plpc': '-0.1304347826086957',
    'usd': None}

In [21]:
# close the option position
trade_client.close_position(
    symbol_or_asset_id = high_open_interest_contract.symbol,
    close_options = ClosePositionRequest(qty = "1")
)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('64a72a00-1651-4db4-a6b0-c4ba6525d3b4'),
    'canceled_at': None,
    'client_order_id': 'c849d15b-3faf-4f75-bc45-33e7df6f54aa',
    'created_at': datetime.datetime(2024, 11, 19, 16, 12, 9, 791381, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('3bd9b604-9d73-477e-8e4b-e32a96b3e09d'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 11, 19, 16, 12, 9, 789782, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY

In [30]:
# exercise the options position
# - this method does not return anything
trade_client.exercise_options_position(
    symbol_or_contract_id = high_open_interest_contract.symbol
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

fyi. you can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [ ]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [ ]:
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                                # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_bars(req).df

In [ ]:
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end=None,                                                     # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_trades(req).df

In [ ]:
# get options exchange codes
option_historical_data_client.get_option_exchange_codes()

In [ ]:
# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

In [ ]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

In [ ]:
# get option snapshot by symbol
req = OptionSnapshotRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_snapshot(req)

In [ ]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

# Market Data (Stream)

In [ ]:
option_data_stream_client = OptionDataStream(api_key, secret_key, url_override = option_stream_data_wss)

async def option_data_stream_handler(data):
    print(data)

symbols = [
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols)
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()